### Импорты

In [1]:
import numpy as np
import pandas as pd

In [2]:
from simpledbf import Dbf5

SQLalchemy is not installed. No support for SQL output.


In [3]:
import matplotlib.pyplot as plt

### Загрузка из dbf (файлы баз данных)

загружаем данные из файлов формата `dbf`  с использованием библиотеки `simpledbf`

In [20]:
groups = Dbf5('Part1/groups.dbf', codec='cp1251')
produces = Dbf5('Part1/produces.dbf', codec='cp866')
sales = Dbf5('Part1/sales.dbf', codec='cp866')
stores = Dbf5('Part1/stores.dbf', codec='cp1251')

конвертируем в нативную для `Pandas` форму датафрейма `df`

In [21]:
groups_df = groups.to_dataframe()
produces_df = produces.to_dataframe()
sales_df = sales.to_dataframe()
stores_df = stores.to_dataframe()

In [22]:
groups_df.head()

,FKGCODE,FKGNAME
0,33,Иммуномодуляторы
1,48,Общетонизирующие средства и адаптогены
2,50,Местные анестетики
3,108,Микро- и макроэлементы
4,198,Витамины и витаминоподобные средства


In [26]:
groups_df.shape

(8, 2)

In [24]:
produces_df.head()

,PRODID,NAME,FKGCODE
0,774,Бетаферон пор.лиоф.д/ин. 9600000 ЕД фл. [с р-л...,33
1,808,Бифидумбактерин пор. для приема внутрь пак.флг...,33
2,809,Бифидумбактерин табл. фл. 20 пач.картон. 10 Ви...,33
3,810,Бифидумбактерин супп.ваг. уп.контурн.яч. 5 пач...,33
4,811,Бифидумбактерин супп.ваг. уп.контурн.яч. 5 пач...,33


In [28]:
produces_df.shape

(5086, 3)

In [25]:
sales_df.head()

,DATE,STOREID,PRODID,HOUR,COUNT,SUM
0,2004-01-01,1,31052,13,1,56.50
1,2004-01-01,1,36259,16,1,72.48
2,2004-01-01,1,40345,15,1,15.84
3,2004-01-01,1,65841,14,1,163.02
4,2004-01-01,1,82545,13,1,173.32


In [37]:
sales_df.shape

(8052, 6)

In [23]:
stores_df.head()

,STOREID,NAME
0,1,Аптека 1
1,2,Аптека 2
2,3,Аптека 3


In [38]:
stores_df.shape

(3, 2)

### Слияние таблиц

Слияние `groups_df` и `produces_df` по `FKGCODE` 

In [31]:
prod_and_groups = pd.merge(left=produces_df, right=groups_df, on='FKGCODE', how='left')

Объединяем таблицы `sales_df` и `stores_df` по `STOREID` 

In [34]:
sales_and_stores = pd.merge(left=sales_df, right=stores_df, on='STOREID', how='left')

Объединяем две получившиеся таблицы по `PRODID`, чтобы получить единую таблицу со всей необходимой информацией по продажам

In [40]:
raw_sales = pd.merge(left=sales_and_stores, right=prod_and_groups, on='PRODID', how='left')

In [43]:
raw_sales.head(10)

,DATE,STOREID,PRODID,HOUR,COUNT,SUM,NAME_x,NAME_y,FKGCODE,FKGNAME
0,2004-01-01,1,31052,13,1,56.50,Аптека 1,Ингалипт аэроз. бал.аэроз.алюм. 30 мл пач.карт...,247,Антисептики и дезинфицирующие средства
1,2004-01-01,1,36259,16,1,72.48,Аптека 1,Компливит табл.п.о. бан.полимерн. 60 пач.карто...,198,Витамины и витаминоподобные средства
2,2004-01-01,1,40345,15,1,15.84,Аптека 1,Линимент бальзамический (по Вишневскому) линим...,247,Антисептики и дезинфицирующие средства
3,2004-01-01,1,65841,14,1,163.02,Аптека 1,Стрепсилс Плюс спрей доз. фл.с дозат. 20 мл ко...,247,Антисептики и дезинфицирующие средства
4,2004-01-01,1,82545,13,1,173.32,Аптека 1,"Арбидол-ЛЭНС табл.п.о. 0,1 г бл. 10 кор. 1 Дал...",33,Иммуномодуляторы
5,2004-01-01,1,83563,13,1,47.95,Аптека 1,Доктор Тайсс Эвкалипт леденцы 50 г Dr. Theiss ...,247,Антисептики и дезинфицирующие средства
6,2004-01-02,1,2291,20,1,276.80,Аптека 1,Кальций-Д_3 Никомед табл.жев. с апельс. фл. 10...,198,Витамины и витаминоподобные средства
7,2004-01-02,1,11291,18,1,86.94,Аптека 1,"Арбидол-ЛЭНС табл.п.о. 0,05 г уп.контурн.яч. 1...",33,Иммуномодуляторы
8,2004-01-02,1,24061,19,1,156.29,Аптека 1,Гриппферон капли наз. 10000 МЕ/мл фл.-кап. 10 ...,33,Иммуномодуляторы
9,2004-01-02,1,27649,15,1,210.50,Аптека 1,Доппельгерц Женьшень Актив эликсир фл.темн.сте...,48,Общетонизирующие средства и адаптогены


In [42]:
raw_sales.to_csv('Part1/raw_sales.csv', header=True, index=False)

### Преобразование даты в pandas.datetime

In [68]:
raw_sales = pd.read_csv('Part1/raw_sales.csv', 
                        parse_dates=['DATE'], 
                        infer_datetime_format=True)

In [69]:
raw_sales.head(10)

,DATE,STOREID,PRODID,HOUR,COUNT,SUM,NAME_x,NAME_y,FKGCODE,FKGNAME
0,2004-01-01,1,31052,13,1,56.50,Аптека 1,Ингалипт аэроз. бал.аэроз.алюм. 30 мл пач.карт...,247,Антисептики и дезинфицирующие средства
1,2004-01-01,1,36259,16,1,72.48,Аптека 1,Компливит табл.п.о. бан.полимерн. 60 пач.карто...,198,Витамины и витаминоподобные средства
2,2004-01-01,1,40345,15,1,15.84,Аптека 1,Линимент бальзамический (по Вишневскому) линим...,247,Антисептики и дезинфицирующие средства
3,2004-01-01,1,65841,14,1,163.02,Аптека 1,Стрепсилс Плюс спрей доз. фл.с дозат. 20 мл ко...,247,Антисептики и дезинфицирующие средства
4,2004-01-01,1,82545,13,1,173.32,Аптека 1,"Арбидол-ЛЭНС табл.п.о. 0,1 г бл. 10 кор. 1 Дал...",33,Иммуномодуляторы
5,2004-01-01,1,83563,13,1,47.95,Аптека 1,Доктор Тайсс Эвкалипт леденцы 50 г Dr. Theiss ...,247,Антисептики и дезинфицирующие средства
6,2004-01-02,1,2291,20,1,276.80,Аптека 1,Кальций-Д_3 Никомед табл.жев. с апельс. фл. 10...,198,Витамины и витаминоподобные средства
7,2004-01-02,1,11291,18,1,86.94,Аптека 1,"Арбидол-ЛЭНС табл.п.о. 0,05 г уп.контурн.яч. 1...",33,Иммуномодуляторы
8,2004-01-02,1,24061,19,1,156.29,Аптека 1,Гриппферон капли наз. 10000 МЕ/мл фл.-кап. 10 ...,33,Иммуномодуляторы
9,2004-01-02,1,27649,15,1,210.50,Аптека 1,Доппельгерц Женьшень Актив эликсир фл.темн.сте...,48,Общетонизирующие средства и адаптогены


In [73]:
raw_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8052 entries, 0 to 8051
Data columns (total 10 columns):
DATE       8052 non-null datetime64[ns]
STOREID    8052 non-null int64
PRODID     8052 non-null int64
HOUR       8052 non-null int64
COUNT      8052 non-null int64
SUM        8052 non-null float64
NAME_x     8052 non-null object
NAME_y     8052 non-null object
FKGCODE    8052 non-null int64
FKGNAME    8052 non-null object
dtypes: datetime64[ns](1), float64(1), int64(5), object(3)
memory usage: 629.1+ KB
